In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import time

# Configuração do Banco
db_engine = create_engine('sqlite:///warehouse_clean.db')
csv_file="vendas_sujas.csv"

In [4]:
print("\n--- ETL: Limpeza com Pandas ---")
start = time.time()
# 1. EXTRACT
df = pd.read_csv(csv_file)
rows_originais = len(df)

# 2. TRANSFORM (Limpeza e Formatação)

# A. Remover Duplicatas (Linhas idênticas)
df.drop_duplicates(subset=['id_venda'], inplace=True)

# B. Tratar Nulos (Remover vendas sem valor ou preencher com 0)
# Aqui vamos assumir que venda sem valor é erro e deve ser removida
df.dropna(subset=['valor'], inplace=True)

# C. Padronização de Texto (Trim e Upper)
# Remove espaços (strip) e converte para maiúsculo (upper)
df['categoria'] = df['categoria'].str.strip().str.upper()

# D. Lógica de Negócio (Filtro > 500 e Imposto)
df_final = df[df['valor'] > 500].copy()
df_final['imposto'] = df_final['valor'] * 0.10

# 3. LOAD
df_final.to_sql('vendas_limpas_etl', db_engine, if_exists='replace', index=False)
end = time.time()

print(f"Tempo: {end-start:.2f} segundos")
print(f"Linhas Originais: {rows_originais}")
print(f"Linhas Após Limpeza: {len(df)}")
print(f"Linhas Finais (Filtro > 500): {len(df_final)}")



--- ETL: Limpeza com Pandas ---
Tempo: 11.18 segundos
Linhas Originais: 1050000
Linhas Após Limpeza: 900000
Linhas Finais (Filtro > 500): 454604


In [ ]:
def limpeza_elt_sql():
    print("\n--- ELT: Limpeza com SQL ---")
    # 1. EXTRACT & LOAD (Raw)
    # Carrega tudo, inclusive a sujeira, para tabela _RAW
    df_raw = pd.read_csv(csv_file)
    df_raw.to_sql('vendas_raw_dirty', db_engine, if_exists='replace', index=False)
    
    # 2. TRANSFORM (SQL faz a limpeza)
    query_limpeza = """
    CREATE TABLE IF NOT EXISTS vendas_limpas_elt AS
    SELECT DISTINCT 
        id_venda,
        data,
        -- Limpeza de Texto: TRIM remove espaços, UPPER padroniza caixa alta
        UPPER(TRIM(categoria)) as categoria,
        valor,
        (valor * 0.10) as imposto
    FROM vendas_raw_dirty
    WHERE 
        valor IS NOT NULL  -- Remove nulos
        AND valor > 500    -- Filtro de negócio
    -- O DISTINCT no início já trata as duplicatas de linhas inteiras
    """
    
    with db_engine.connect() as conn:
        conn.execute(text("DROP TABLE IF EXISTS vendas_limpas_elt"))
        conn.execute(text(query_limpeza))
        conn.commit() # Importante para efetivar a criação
        
        # Contagem para validação
        count = conn.execute(text("SELECT COUNT(*) FROM vendas_limpas_elt")).scalar()
        print(f"Linhas Finais na tabela ELT: {count}")
    
start=time.time()
limpeza_elt_sql()
end=time.time()
print(f"Tempo: {end-start:.2f} segundos")


--- ELT: Limpeza com SQL ---
Linhas Finais na tabela ELT: 454604
Tempo: 22.23 segundos
